# 1 国家统计局数据爬虫（动态网页爬虫）

## 1.1 单个价格指标的获取

In [1]:
#导入爬虫需要的包
import time
import requests
from bs4 import BeautifulSoup as bs

#不显示警示信息(不影响程序运行, 也可注释掉这两句)
import warnings
warnings.filterwarnings("ignore")

#爬虫请求头的填充
headers = {
    "Accept-Language": "zh-cn",
    "Host": "data.stats.gov.cn",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.2 Safari/605.1.15",
    "Referer": "https://data.stats.gov.cn/easyquery.htm?cn=A01",
    "Accept-Encoding": "br, gzip, deflate"
}

#爬虫请求参数的填充
keyvalue ={
    'm': 'QueryData',
    'dbcode': 'hgyd',
    'rowcode': 'zb',
    'colcode': 'sj',
    'wds': '[]',
    'dfwds': '[{"wdcode":"zb","valuecode":"A010101"}]',
    'k1': '1621085284011',
    #也可写成——'k1': time.time(),
    'h': '1'
}

#设置网站地址, 获取网站信息
url="https://data.stats.gov.cn/easyquery.htm?cn=A01"

#对于统计局网站, 必须要设置verify=False选项, 否则无法打开页面
res = requests.get(url, headers = headers, params = keyvalue, verify = False) 

soup=bs(res.content,"lxml")


#解析网页信息
import json
import numpy as np
import pandas as pd

txt = soup.get_text()
j = json.loads(txt)

#解析json格式的数据
rowlist = [elem['name'] for elem in j['returndata']['wdnodes'][0]['nodes']]
collist = [elem['name'] for elem in j['returndata']['wdnodes'][1]['nodes']]
data = [elem['data']['data'] for elem in j['returndata']['datanodes']]
ndata=np.asarray(data)

lenOfRow = len(rowlist)
lenOfCol = len(collist)
rdata = ndata.reshape(lenOfRow,lenOfCol)

#将数据用pandas包整理成DataFrame格式
df = pd.DataFrame(data = rdata, index = rowlist, columns = collist)

df

,2022年2月,2022年1月,2021年12月,2021年11月,2021年10月,2021年9月,2021年8月,2021年7月,2021年6月,2021年5月,2021年4月,2021年3月,2021年2月
居民消费价格指数(上年同月=100),100.9,100.9,101.5,102.3,101.5,100.7,100.8,101.0,101.1,101.3,100.9,100.4,99.8
食品烟酒类居民消费价格指数(上年同月=100),98.2,98.2,99.9,101.7,99.1,97.2,98.0,98.2,99.6,100.8,100.1,100.1,100.3
衣着类居民消费价格指数(上年同月=100),100.6,100.4,100.6,100.5,100.5,100.5,100.5,100.4,100.4,100.4,100.2,100.1,99.5
居住类居民消费价格指数(上年同月=100),101.4,101.4,101.6,101.7,101.7,101.3,101.1,101.1,100.9,100.7,100.4,100.2,99.7
生活用品及服务类居民消费价格指数(上年同月=100),100.6,100.4,100.8,100.5,100.9,100.5,100.6,100.3,100.3,100.4,100.4,100.0,99.8
交通和通信类居民消费价格指数(上年同月=100),105.5,105.2,105.0,107.6,107.0,105.8,105.9,106.9,105.8,105.5,104.9,102.7,98.1
教育文化和娱乐类居民消费价格指数(上年同月=100),102.5,102.9,103.1,103.0,102.9,103.2,103.0,102.7,101.5,101.5,101.3,100.4,100.6
医疗保健类居民消费价格指数(上年同月=100),100.6,100.6,100.7,100.6,100.6,100.4,100.4,100.4,100.3,100.2,100.1,100.2,100.3
其他用品和服务类居民消费价格指数(上年同月=100),100.3,99.9,99.5,99.4,99.5,97.2,96.1,98.7,99.1,99.1,98.7,98.5,99.2


## 1.2 循环获取所有价格指标

In [2]:
#导入爬虫需要的包
import time
import requests
from bs4 import BeautifulSoup as bs

#生成每一张数据表专属的valuecode
#网站https://data.stats.gov.cn/easyquery.htm?cn=A01
#chrome——右击（检查）——Network——Feth｜XHR——点击页面上的指标（例如: 全国居民消费价格指数）——Headers选项中的Request URL
#https://data.stats.gov.cn/easyquery.htm?m=QueryData&dbcode=hgyd&rowcode=zb&colcode=sj&wds=[]&dfwds=[{"wdcode":"zb","valuecode":"A010101"}]&k1=1647148039314&h=1
#通过不断切换数据表发现——变化的是"valuecode":"A010101"这一部分, 因此可以模仿生成这些code, 加入URL便可获取数据

code_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G']

#仅获取第一张数据表, 一次性获取太多容易被封ip, 若要获取全部的信息（要么降低访问频率, 要么使用代理ip）

for each in code_list[:1]:
    
    for item in code_list[1:]:
    
        valuecode = 'A010' + each + '0' + item
    
        #爬虫请求头的填充
        headers = {
            "Accept-Language": "zh-cn",
            "Host": "data.stats.gov.cn",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.2 Safari/605.1.15",
            "Referer": "https://data.stats.gov.cn/easyquery.htm?cn=A01",
            "Accept-Encoding": "br, gzip, deflate"
        }

        #爬虫请求参数的填充
        keyvalue ={
            'm': 'QueryData',
            'dbcode': 'hgyd',
            'rowcode': 'zb',
            'colcode': 'sj',
            'wds': '[]',
            'dfwds': str([{"wdcode": "zb", "valuecode": valuecode}]).replace("'",'"'),
            'k1': '1621085284011',
            'h': '1'
        }

        #设置网站地址, 获取网站信息
        url="https://data.stats.gov.cn/easyquery.htm?cn=A01"

        #对于统计局网站, 必须要设置verify=False选项, 否则无法打开页面(运行时会出现红色提示信息, 但不会影响程序运行)
        res = requests.get(url, headers = headers, params = keyvalue, verify = False) 

        soup = bs(res.content,"lxml")

        if '对不起' not in soup.get_text():

            print('检测到如下数据编号: ', end = ' ')
            print(valuecode, end = ' ')
            
            try:
                #解析网页信息
                import json
                import numpy as np
                import pandas as pd

                txt = soup.get_text()
                j = json.loads(txt)

                #解析json格式的数据
                rowlist = [elem['name'] for elem in j['returndata']['wdnodes'][0]['nodes']]
                collist = [elem['name'] for elem in j['returndata']['wdnodes'][1]['nodes']]
                data = [elem['data']['data'] for elem in j['returndata']['datanodes']]
                ndata = np.asarray(data)

                lenOfRow = len(rowlist)
                lenOfCol = len(collist)
                rdata = ndata.reshape(lenOfRow,lenOfCol)

                #将数据用pandas包整理成DataFrame格式
                df = pd.concat([df, pd.DataFrame(data = rdata, index = rowlist, columns = collist)], axis = 0)

                print('...爬取完成')
            
            except:
                print('...爬取失败')
            
        #若爬虫访问速度过快, 被系统检测到后会封禁一段时间, 则增加访问的间隔
        #time.sleep(2)
            
        if '请开启JavaScript并刷新该页' in soup.get_text():
            
            print('爬虫速度过快, 被系统检测到, 请降低访问频率（或更换ip访问）！')

#输出爬虫结果
stat_data = df.drop_duplicates()
stat_data

检测到如下数据编号:  A010102 ...爬取完成
检测到如下数据编号:  A010103 ...爬取完成
检测到如下数据编号:  A010104 ...爬取完成
检测到如下数据编号:  A010105 ...爬取完成
检测到如下数据编号:  A010106 ...爬取完成
检测到如下数据编号:  A010107 ...爬取完成
检测到如下数据编号:  A010108 ...爬取完成
检测到如下数据编号:  A010109 ...爬取完成


,2022年2月,2022年1月,2021年12月,2021年11月,2021年10月,2021年9月,2021年8月,2021年7月,2021年6月,2021年5月,...,2016年6月,2016年5月,2016年4月,2016年3月,2016年2月,2016年1月,2015年12月,2015年11月,2015年10月,2015年9月
居民消费价格指数(上年同月=100),100.9,100.9,101.5,102.3,101.5,100.7,100.8,101.0,101.1,101.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
食品烟酒类居民消费价格指数(上年同月=100),98.2,98.2,99.9,101.7,99.1,97.2,98.0,98.2,99.6,100.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
衣着类居民消费价格指数(上年同月=100),100.6,100.4,100.6,100.5,100.5,100.5,100.5,100.4,100.4,100.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
居住类居民消费价格指数(上年同月=100),101.4,101.4,101.6,101.7,101.7,101.3,101.1,101.1,100.9,100.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
生活用品及服务类居民消费价格指数(上年同月=100),100.6,100.4,100.8,100.5,100.9,100.5,100.6,100.3,100.3,100.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
畜肉类农村居民消费价格指数(上年同月=100),70.6,70.7,74.9,77.9,69.9,67.6,69.0,70.6,76.7,86.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
蛋类农村居民消费价格指数(上年同月=100),102.2,101.5,113.3,119.1,113.9,113.8,115.6,118.0,120.1,116.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
水产品类农村居民消费价格指数(上年同月=100),106.4,109.1,108.6,109.8,110.1,111.7,114.3,116.3,116.5,116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
鲜菜类农村居民消费价格指数(上年同月=100),101.0,95.8,111.1,129.9,114.1,98.1,98.1,95.6,100.5,104.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.3 课后练习

- 练习1: 现在只能获取到近12个月的指标, 尝试获取到36个月的指标
- 练习2: 试着将爬取的数据整理写入Excel表格中（提示: 使用pandas）
- 练习3: 找出练习2中数据的错误, 并修正（提示: 空值, 修改变量名）
- 思考1: 静态与动态网页的不同, 爬取时的差异在哪里?